https://github.com/susanli2016/Machine-Learning-with-Python

In [ ]:
import csv
%load_ext autoreload
%autoreload 2
from progress import ProgressTracker

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [ ]:
import nltk
nltk.download('wordnet')

In [ ]:
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [ ]:
for w in ['dogs', 'ran', 'discouraged']:
    print(w, get_lemma(w), get_lemma2(w))

In [ ]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

In [ ]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [ ]:
import random
text_data = []
with open('data/dataset.csv') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        if random.random() > .99:
            print(tokens)
            text_data.append(tokens)

In [ ]:
maxTweets = 4000000
tracker = ProgressTracker(maxTweets)
full_texts = []
twitter_accounts = []
with open('data/tweetsDFMin.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row, line_count in zip(csv_reader, range(maxTweets)):
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
        else:
            try:
                full_texts.append(row[1])
            except IndexError:
                continue
            try:
                twitter_accounts.append(row[2])
            except IndexError:
                full_texts.pop()
                pass
        tracker.update(line_count)
    print(f'Processed {line_count} lines.')
    print(f'Got {len(full_texts)} tweets')
    print(f'Got {len(twitter_accounts)} account ids')

In [ ]:
tweetsDF = pd.DataFrame({"full_text": full_texts, "twitter_account": twitter_accounts})

In [ ]:
tweetsDF.to_csv("data/tweetsDFMinReadable.csv", index=False)

In [ ]:
tweetsDF = pd.read_csv("data/tweetsDFMinReadable.csv")

In [ ]:
tweetsDF

In [ ]:
maxTweets = 100000
rawTweets = tweetsDF.loc[:maxTweets,"full_text"]

## Or, choosing to treat tweet sets as documents

In [ ]:
rawTweets = tweetsDF.groupby('twitter_account')['full_text'].apply(lambda x: "%s" % ' --\n '.join(x))

In [ ]:
# preppedTweets = [prepare_text_for_lda(text) for text in rawTweets]

In [ ]:
tracker = ProgressTracker(len(rawTweets), 5)
preppedTweets = []
for i, text in enumerate(rawTweets):
    preppedTweets.append(prepare_text_for_lda(text))
    tracker.update(i)

In [ ]:
text_data = preppedTweets

In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)

In [33]:
# Set up log to terminal, per https://miningthedetails.com/blog/python/lda/GensimLDA/
import logging
logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
logging.info("hi")

In [ ]:
corpus = [dictionary.doc2bow(text) for text in text_data]

In [ ]:
import pickle
pickle.dump(corpus, open('data/corpus.pkl', 'wb'))
dictionary.save('data/dictionary.gensim')

### Try 20 topics

In [ ]:
import gensim
NUM_TOPICS = 20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('LDAModels/model20.gensim')

In [35]:
topics = ldamodel.print_topics(num_words=5)
for topic in topics:
    print(topic)

2019-02-25 17:22:46,804 : INFO : topic #0 (0.050): 0.192*"SCREEN_NAME" + 0.049*"think" + 0.042*"could" + 0.025*"committee" + 0.024*"program"
2019-02-25 17:22:46,808 : INFO : topic #1 (0.050): 0.052*"community" + 0.038*"working" + 0.034*"issue" + 0.031*"clinton" + 0.030*"pass"
2019-02-25 17:22:46,812 : INFO : topic #2 (0.050): 0.036*"fight" + 0.035*"always" + 0.026*"colleague" + 0.025*"vote" + 0.024*"foreign"
2019-02-25 17:22:46,817 : INFO : topic #3 (0.050): 0.173*"SCREEN_NAME" + 0.058*"tonight" + 0.035*"friend" + 0.032*"better" + 0.032*"country"
2019-02-25 17:22:46,820 : INFO : topic #4 (0.050): 0.454*"future" + 0.043*"woman" + 0.034*"congress" + 0.034*"protect" + 0.031*"change"
2019-02-25 17:22:46,824 : INFO : topic #5 (0.050): 0.308*"class" + 0.157*"SCREEN_NAME" + 0.068*"today" + 0.042*"first" + 0.031*"going"
2019-02-25 17:22:46,828 : INFO : topic #6 (0.050): 0.057*"republican" + 0.042*"every" + 0.037*"leaders" + 0.034*"attack" + 0.028*"debate"
2019-02-25 17:22:46,831 : INFO : topic

(0, '0.192*"SCREEN_NAME" + 0.049*"think" + 0.042*"could" + 0.025*"committee" + 0.024*"program"')
(1, '0.052*"community" + 0.038*"working" + 0.034*"issue" + 0.031*"clinton" + 0.030*"pass"')
(2, '0.036*"fight" + 0.035*"always" + 0.026*"colleague" + 0.025*"vote" + 0.024*"foreign"')
(3, '0.173*"SCREEN_NAME" + 0.058*"tonight" + 0.035*"friend" + 0.032*"better" + 0.032*"country"')
(4, '0.454*"future" + 0.043*"woman" + 0.034*"congress" + 0.034*"protect" + 0.031*"change"')
(5, '0.308*"class" + 0.157*"SCREEN_NAME" + 0.068*"today" + 0.042*"first" + 0.031*"going"')
(6, '0.057*"republican" + 0.042*"every" + 0.037*"leaders" + 0.034*"attack" + 0.028*"debate"')
(7, '0.046*"hearing" + 0.039*"bring" + 0.039*"congratulations" + 0.031*"district" + 0.030*"increase"')
(8, '0.180*"SCREEN_NAME" + 0.038*"story" + 0.036*"report" + 0.035*"start" + 0.030*"coming"')
(9, '0.096*"house" + 0.077*"right" + 0.047*"america" + 0.026*"white" + 0.026*"former"')
(10, '0.439*"enjoy" + 0.026*"public" + 0.024*"meeting" + 0.021

In [49]:
ldamodel = gensim.models.ldamodel.LdaModel.load("LDAModels/model20.gensim")

2019-02-25 17:27:12,450 : INFO : loading LdaModel object from LDAModels/model20.gensim
2019-02-25 17:27:12,453 : INFO : loading expElogbeta from LDAModels/model20.gensim.expElogbeta.npy with mmap=None
2019-02-25 17:27:12,456 : INFO : setting ignored attribute state to None
2019-02-25 17:27:12,457 : INFO : setting ignored attribute id2word to None
2019-02-25 17:27:12,458 : INFO : setting ignored attribute dispatcher to None
2019-02-25 17:27:12,458 : INFO : loaded LDAModels/model20.gensim
2019-02-25 17:27:12,459 : INFO : loading LdaState object from LDAModels/model20.gensim.state
2019-02-25 17:27:12,491 : INFO : loaded LDAModels/model20.gensim.state


In [50]:
i = 4
print(ldamodel.get_document_topics(corpus[i]))
print(text_data[i])

[(0, 0.29302165), (1, 0.15009089), (4, 0.15009674), (10, 0.2924213)]
['SCREEN_NAME', 'advise', 'throw', 'romaine', 'lettuce', 'already', 'eat', 'get']


In [51]:
tweetsDF.loc[i,"full_text"]

'RT @NYMag: The CDC advises to throw out any and all romaine lettuce, even if you have already eaten some of it and have not gotten sick htt…'

In [52]:
for NUM_TOPICS in (5, 10):
    print(NUM_TOPICS)
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
    ldamodel.save(f'LDAModels/model{NUM_TOPICS}.gensim')

2019-02-25 17:29:04,325 : INFO : using symmetric alpha at 0.2
2019-02-25 17:29:04,326 : INFO : using symmetric eta at 0.2
2019-02-25 17:29:04,338 : INFO : using serial LDA version on this node
2019-02-25 17:29:04,382 : INFO : running online (multi-pass) LDA training, 5 topics, 15 passes over the supplied corpus of 100001 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2019-02-25 17:29:04,384 : INFO : PROGRESS: pass 0, at document #2000/100001


5


2019-02-25 17:29:05,828 : INFO : merging changes from 2000 documents into a model of 100001 documents
2019-02-25 17:29:05,845 : INFO : topic #0 (0.200): 0.114*"SCREEN_NAME" + 0.015*"trump" + 0.011*"morning" + 0.008*"thank" + 0.008*"happy" + 0.006*"donate" + 0.006*"today" + 0.005*"american" + 0.005*"relief" + 0.005*"philippines"
2019-02-25 17:29:05,847 : INFO : topic #1 (0.200): 0.117*"SCREEN_NAME" + 0.011*"trump" + 0.007*"would" + 0.007*"people" + 0.005*"washington" + 0.005*"today" + 0.005*"think" + 0.005*"airport" + 0.005*"givingusa" + 0.004*"story"
2019-02-25 17:29:05,849 : INFO : topic #2 (0.200): 0.111*"SCREEN_NAME" + 0.006*"people" + 0.006*"things" + 0.005*"health" + 0.005*"trump" + 0.004*"world" + 0.004*"giving" + 0.004*"grateful" + 0.004*"community" + 0.004*"president"
2019-02-25 17:29:05,851 : INFO : topic #3 (0.200): 0.223*"SCREEN_NAME" + 0.010*"thanks" + 0.008*"trump" + 0.006*"people" + 0.006*"today" + 0.005*"woman" + 0.004*"great" + 0.004*"happy" + 0.004*"white" + 0.003*"ton

2019-02-25 17:29:12,230 : INFO : merging changes from 2000 documents into a model of 100001 documents
2019-02-25 17:29:12,255 : INFO : topic #0 (0.200): 0.089*"SCREEN_NAME" + 0.016*"energy" + 0.014*"thank" + 0.007*"electric" + 0.007*"great" + 0.007*"state" + 0.006*"decade" + 0.006*"fossil" + 0.006*"ocean" + 0.005*"today"
2019-02-25 17:29:12,259 : INFO : topic #1 (0.200): 0.098*"SCREEN_NAME" + 0.038*"climate" + 0.009*"plant" + 0.007*"business" + 0.007*"would" + 0.006*"trump" + 0.006*"major" + 0.006*"show" + 0.006*"issue" + 0.005*"important"
2019-02-25 17:29:12,262 : INFO : topic #2 (0.200): 0.077*"SCREEN_NAME" + 0.014*"natural" + 0.010*"nuclear" + 0.010*"future" + 0.009*"voter" + 0.008*"health" + 0.007*"global" + 0.007*"study" + 0.006*"people" + 0.006*"political"
2019-02-25 17:29:12,267 : INFO : topic #3 (0.200): 0.223*"SCREEN_NAME" + 0.008*"great" + 0.007*"state" + 0.006*"please" + 0.005*"today" + 0.005*"thanks" + 0.005*"people" + 0.005*"change" + 0.004*"scientist" + 0.004*"industry"
2

2019-02-25 17:29:18,812 : INFO : topic diff=0.229091, rho=0.288675
2019-02-25 17:29:18,814 : INFO : PROGRESS: pass 0, at document #26000/100001
2019-02-25 17:29:19,940 : INFO : merging changes from 2000 documents into a model of 100001 documents
2019-02-25 17:29:19,957 : INFO : topic #0 (0.200): 0.070*"SCREEN_NAME" + 0.026*"thank" + 0.010*"morning" + 0.009*"veteran" + 0.009*"great" + 0.009*"today" + 0.008*"energy" + 0.008*"working" + 0.008*"honor" + 0.007*"discus"
2019-02-25 17:29:19,962 : INFO : topic #1 (0.200): 0.077*"SCREEN_NAME" + 0.009*"business" + 0.009*"local" + 0.008*"climate" + 0.007*"technology" + 0.007*"senate" + 0.007*"today" + 0.007*"cybersecurity" + 0.006*"proud" + 0.006*"security"
2019-02-25 17:29:19,964 : INFO : topic #2 (0.200): 0.063*"SCREEN_NAME" + 0.022*"texas" + 0.012*"future" + 0.012*"community" + 0.010*"member" + 0.008*"celebrate" + 0.007*"health" + 0.007*"right" + 0.006*"problem" + 0.006*"things"
2019-02-25 17:29:19,966 : INFO : topic #3 (0.200): 0.221*"SCREEN_

...

2019-02-25 17:35:54,457 : INFO : topic diff=0.078684, rho=0.127000
2019-02-25 17:35:54,459 : INFO : PROGRESS: pass 11, at document #90000/100001
2019-02-25 17:35:54,949 : INFO : merging changes from 2000 documents into a model of 100001 documents
2019-02-25 17:35:54,965 : INFO : topic #0 (0.200): 0.030*"thank" + 0.017*"great" + 0.015*"honor" + 0.013*"ocean" + 0.012*"community" + 0.011*"working" + 0.009*"morning" + 0.008*"service" + 0.008*"county" + 0.008*"veteran"
2019-02-25 17:35:54,967 : INFO : topic #1 (0.200): 0.022*"trump" + 0.019*"president" + 0.018*"american" + 0.012*"republican" + 0.010*"senate" + 0.009*"house" + 0.008*"protect" + 0.008*"would" + 0.007*"attack" + 0.007*"security"
2019-02-25 17:35:54,970 : INFO : topic #2 (0.200): 0.027*"onthisday" + 0.019*"health" + 0.013*"member" + 0.012*"future" + 0.009*"right" + 0.007*"animal" + 0.007*"american" + 0.006*"first" + 0.006*"human" + 0.006*"things"
2019-02-25 17:35:54,972 : INFO : topic #3 (0.200): 0.277*"SCREEN_NAME" + 0.014*"to

2019-02-25 17:35:58,433 : INFO : topic diff=0.171613, rho=0.127000
2019-02-25 17:35:58,463 : INFO : -5.490 per-word bound, 45.0 perplexity estimate based on a held-out corpus of 1 documents with 10 words
2019-02-25 17:35:58,464 : INFO : PROGRESS: pass 11, at document #100001/100001
2019-02-25 17:35:58,481 : INFO : merging changes from 1 documents into a model of 100001 documents
2019-02-25 17:35:58,506 : INFO : topic #0 (0.200): 0.024*"thank" + 0.017*"great" + 0.014*"morning" + 0.013*"honor" + 0.010*"community" + 0.009*"working" + 0.009*"service" + 0.008*"ocean" + 0.007*"state" + 0.007*"trade"
2019-02-25 17:35:58,509 : INFO : topic #1 (0.200): 0.031*"trump" + 0.016*"president" + 0.013*"american" + 0.009*"republican" + 0.009*"house" + 0.007*"senate" + 0.006*"would" + 0.006*"security" + 0.006*"sobre" + 0.006*"obama"
2019-02-25 17:35:58,515 : INFO : topic #2 (0.200): 0.136*"future" + 0.011*"onthisday" + 0.010*"shelton" + 0.010*"member" + 0.008*"health" + 0.007*"chaffetz" + 0.005*"right" +

2019-02-25 17:36:01,280 : INFO : topic diff=0.102627, rho=0.125988
2019-02-25 17:36:01,281 : INFO : PROGRESS: pass 12, at document #12000/100001
2019-02-25 17:36:01,829 : INFO : merging changes from 2000 documents into a model of 100001 documents
2019-02-25 17:36:01,845 : INFO : topic #0 (0.200): 0.023*"thank" + 0.015*"great" + 0.011*"morning" + 0.011*"community" + 0.011*"honor" + 0.009*"working" + 0.008*"county" + 0.008*"energy" + 0.008*"state" + 0.007*"veteran"
2019-02-25 17:36:01,848 : INFO : topic #1 (0.200): 0.021*"trump" + 0.010*"president" + 0.010*"business" + 0.009*"climate" + 0.008*"american" + 0.008*"orleg" + 0.007*"would" + 0.007*"people" + 0.007*"house" + 0.006*"election"
2019-02-25 17:36:01,850 : INFO : topic #2 (0.200): 0.073*"future" + 0.014*"health" + 0.009*"member" + 0.007*"things" + 0.007*"right" + 0.006*"point" + 0.006*"problem" + 0.006*"political" + 0.005*"natural" + 0.005*"onthisday"
2019-02-25 17:36:01,853 : INFO : topic #3 (0.200): 0.304*"SCREEN_NAME" + 0.024*"to

2019-02-25 17:36:05,438 : INFO : topic diff=0.088426, rho=0.125988
2019-02-25 17:36:05,440 : INFO : PROGRESS: pass 12, at document #24000/100001
2019-02-25 17:36:05,987 : INFO : merging changes from 2000 documents into a model of 100001 documents
2019-02-25 17:36:06,004 : INFO : topic #0 (0.200): 0.024*"thank" + 0.018*"community" + 0.014*"great" + 0.014*"honor" + 0.012*"energy" + 0.010*"morning" + 0.008*"working" + 0.008*"county" + 0.008*"service" + 0.008*"state"
2019-02-25 17:36:06,007 : INFO : topic #1 (0.200): 0.019*"trump" + 0.012*"climate" + 0.011*"american" + 0.010*"president" + 0.009*"business" + 0.006*"people" + 0.006*"senate" + 0.006*"would" + 0.006*"change" + 0.006*"security"
2019-02-25 17:36:06,009 : INFO : topic #2 (0.200): 0.041*"future" + 0.013*"health" + 0.011*"member" + 0.010*"texas" + 0.010*"problem" + 0.008*"things" + 0.008*"right" + 0.006*"natural" + 0.006*"political" + 0.006*"point"
2019-02-25 17:36:06,012 : INFO : topic #3 (0.200): 0.288*"SCREEN_NAME" + 0.019*"toda

KeyboardInterrupt: 

In [53]:
maxTweets

4000000

In [58]:
maxTweets = 100000
for doGroup in (False, True):
    print(f'Start doGroup={doGroup}')
    if doGroup:
        # 500 docs by long
        rawTweets = tweetsDF[:maxTweets*10].groupby('twitter_account')['full_text'].apply(lambda x: "%s" % ' --\n '.join(x))
    else:
        # maxTweets docs by short
        rawTweets = tweetsDF.loc[:maxTweets,"full_text"]

    preppedTweets = []
    for i, text in enumerate(rawTweets):
        preppedTweets.append(prepare_text_for_lda(text))

    text_data = preppedTweets
    dictionary = corpora.Dictionary(text_data)
    corpus = [dictionary.doc2bow(text) for text in text_data]

    for NUM_TOPICS in (5, 10, 20):
        if doGroup == False and NUM_TOPICS == 20:
            break
        print(doGroup, NUM_TOPICS)
        ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
        ldamodel.save(f'LDAModels/model{NUM_TOPICS}.grouped={doGroup}.gensim')

Start doGroup=False


2019-02-25 17:48:25,657 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-02-25 17:48:25,863 : INFO : adding document #10000 to Dictionary(11498 unique tokens: ['1)some', 'SCREEN_NAME', 'book', 'early', 'fiction']...)
2019-02-25 17:48:26,077 : INFO : adding document #20000 to Dictionary(18512 unique tokens: ['1)some', 'SCREEN_NAME', 'book', 'early', 'fiction']...)
2019-02-25 17:48:26,280 : INFO : adding document #30000 to Dictionary(23099 unique tokens: ['1)some', 'SCREEN_NAME', 'book', 'early', 'fiction']...)
2019-02-25 17:48:26,493 : INFO : adding document #40000 to Dictionary(28113 unique tokens: ['1)some', 'SCREEN_NAME', 'book', 'early', 'fiction']...)
2019-02-25 17:48:26,712 : INFO : adding document #50000 to Dictionary(32368 unique tokens: ['1)some', 'SCREEN_NAME', 'book', 'early', 'fiction']...)
2019-02-25 17:48:26,936 : INFO : adding document #60000 to Dictionary(37296 unique tokens: ['1)some', 'SCREEN_NAME', 'book', 'early', 'fiction']...)
2019-02-25 17:48:27

False 5


2019-02-25 17:48:30,901 : INFO : merging changes from 2000 documents into a model of 100001 documents
2019-02-25 17:48:30,939 : INFO : topic #0 (0.200): 0.189*"SCREEN_NAME" + 0.013*"trump" + 0.008*"today" + 0.005*"maybe" + 0.005*"people" + 0.004*"thing" + 0.004*"think" + 0.004*"could" + 0.004*"friend" + 0.004*"right"
2019-02-25 17:48:30,942 : INFO : topic #1 (0.200): 0.097*"SCREEN_NAME" + 0.005*"could" + 0.005*"anyone" + 0.005*"world" + 0.005*"great" + 0.005*"still" + 0.005*"social" + 0.004*"right" + 0.004*"debate" + 0.004*"process"
2019-02-25 17:48:30,944 : INFO : topic #2 (0.200): 0.190*"SCREEN_NAME" + 0.010*"trump" + 0.006*"woman" + 0.005*"today" + 0.005*"white" + 0.005*"right" + 0.004*"would" + 0.004*"happy" + 0.004*"thanks" + 0.003*"president"
2019-02-25 17:48:30,946 : INFO : topic #3 (0.200): 0.058*"SCREEN_NAME" + 0.013*"trump" + 0.011*"white" + 0.007*"woman" + 0.007*"happen" + 0.006*"health" + 0.005*"fundraise" + 0.004*"charity" + 0.004*"water" + 0.004*"donate"
2019-02-25 17:48:

2019-02-25 17:48:37,401 : INFO : merging changes from 2000 documents into a model of 100001 documents
2019-02-25 17:48:37,422 : INFO : topic #0 (0.200): 0.203*"SCREEN_NAME" + 0.009*"fossil" + 0.007*"trump" + 0.006*"plant" + 0.006*"nuclear" + 0.006*"today" + 0.006*"fuel" + 0.005*"thank" + 0.005*"power" + 0.005*"people"
2019-02-25 17:48:37,426 : INFO : topic #1 (0.200): 0.071*"SCREEN_NAME" + 0.008*"reduce" + 0.008*"court" + 0.008*"state" + 0.007*"could" + 0.007*"kavanaugh" + 0.006*"still" + 0.006*"ocean" + 0.006*"change" + 0.006*"project"
2019-02-25 17:48:37,429 : INFO : topic #2 (0.200): 0.161*"SCREEN_NAME" + 0.019*"climate" + 0.010*"change" + 0.007*"industry" + 0.006*"energy" + 0.006*"global" + 0.006*"would" + 0.006*"scientist" + 0.005*"trump" + 0.005*"today"
2019-02-25 17:48:37,434 : INFO : topic #3 (0.200): 0.043*"SCREEN_NAME" + 0.026*"oregon" + 0.017*"state" + 0.013*"pipeline" + 0.011*"health" + 0.010*"governor" + 0.010*"large" + 0.009*"carbon" + 0.009*"county" + 0.009*"study"
2019-

...

2019-02-25 18:33:07,483 : INFO : topic #8 (0.100): 0.108*"SCREEN_NAME" + 0.013*"today" + 0.007*"great" + 0.006*"house" + 0.006*"american" + 0.005*"thank" + 0.005*"thanks" + 0.005*"family" + 0.004*"support" + 0.004*"congress"
2019-02-25 18:33:07,488 : INFO : topic #1 (0.100): 0.062*"SCREEN_NAME" + 0.008*"sobre" + 0.006*"nigeria" + 0.003*"venezuela" + 0.003*"trump" + 0.003*"africa" + 0.003*"maduro" + 0.003*"contra" + 0.003*"ocean" + 0.002*"oceana"
2019-02-25 18:33:07,494 : INFO : topic diff=0.071307, rho=0.277350
2019-02-25 18:33:22,871 : INFO : -8.117 per-word bound, 277.7 perplexity estimate based on a held-out corpus of 449 documents with 8187854 words
2019-02-25 18:33:22,872 : INFO : PROGRESS: pass 12, at document #449/449
2019-02-25 18:33:25,727 : INFO : topic #3 (0.100): 0.019*"newton" + 0.015*"representative" + 0.009*"SCREEN_NAME" + 0.007*"enjoy" + 0.006*"florida" + 0.006*"current" + 0.006*"riley" + 0.006*"situation" + 0.005*"petersburg" + 0.005*"pinellas"
2019-02-25 18:33:25,732 

True 20


2019-02-25 18:34:03,488 : INFO : running online (multi-pass) LDA training, 20 topics, 15 passes over the supplied corpus of 449 documents, updating model once every 449 documents, evaluating perplexity every 449 documents, iterating 50x with a convergence threshold of 0.001000
2019-02-25 18:34:20,268 : INFO : -14.219 per-word bound, 19065.3 perplexity estimate based on a held-out corpus of 449 documents with 8187854 words
2019-02-25 18:34:20,269 : INFO : PROGRESS: pass 0, at document #449/449
2019-02-25 18:34:23,942 : INFO : topic #4 (0.050): 0.085*"SCREEN_NAME" + 0.009*"today" + 0.008*"great" + 0.004*"house" + 0.004*"american" + 0.004*"thank" + 0.004*"support" + 0.004*"family" + 0.004*"thanks" + 0.003*"people"
2019-02-25 18:34:23,948 : INFO : topic #13 (0.050): 0.086*"SCREEN_NAME" + 0.007*"today" + 0.005*"american" + 0.005*"trump" + 0.004*"house" + 0.004*"great" + 0.004*"family" + 0.003*"support" + 0.003*"right" + 0.003*"would"
2019-02-25 18:34:23,954 : INFO : topic #15 (0.050): 0.163

2019-02-25 18:36:04,042 : INFO : topic #18 (0.050): 0.098*"SCREEN_NAME" + 0.011*"cancer" + 0.006*"sobre" + 0.004*"breast" + 0.003*"trump" + 0.003*"patient" + 0.003*"venezuela" + 0.003*"today" + 0.002*"maduro" + 0.002*"great"
2019-02-25 18:36:04,048 : INFO : topic #7 (0.050): 0.035*"SCREEN_NAME" + 0.003*"today" + 0.002*"great" + 0.002*"american" + 0.002*"family" + 0.002*"house" + 0.002*"support" + 0.002*"thank" + 0.002*"congress" + 0.001*"people"
2019-02-25 18:36:04,055 : INFO : topic diff=0.573259, rho=0.377964
2019-02-25 18:36:20,285 : INFO : -8.310 per-word bound, 317.3 perplexity estimate based on a held-out corpus of 449 documents with 8187854 words
2019-02-25 18:36:20,286 : INFO : PROGRESS: pass 6, at document #449/449
2019-02-25 18:36:24,098 : INFO : topic #2 (0.050): 0.118*"SCREEN_NAME" + 0.008*"climate" + 0.006*"great" + 0.006*"energy" + 0.006*"trump" + 0.006*"today" + 0.004*"thank" + 0.004*"change" + 0.004*"clean" + 0.003*"water"
2019-02-25 18:36:24,103 : INFO : topic #6 (0.05

2019-02-25 18:38:04,652 : INFO : topic #15 (0.050): 0.217*"SCREEN_NAME" + 0.006*"great" + 0.005*"today" + 0.004*"thanks" + 0.004*"thank" + 0.003*"people" + 0.003*"woman" + 0.003*"first" + 0.003*"world" + 0.002*"change"
2019-02-25 18:38:04,656 : INFO : topic #9 (0.050): 0.025*"SCREEN_NAME" + 0.021*"idaho" + 0.021*"idpol" + 0.020*"nigeria" + 0.010*"africa" + 0.010*"photo" + 0.010*"mark" + 0.006*"south" + 0.005*"security" + 0.004*"buck"
2019-02-25 18:38:04,661 : INFO : topic #16 (0.050): 0.001*"SCREEN_NAME" + 0.000*"today" + 0.000*"great" + 0.000*"vicksburg" + 0.000*"house" + 0.000*"state" + 0.000*"american" + 0.000*"community" + 0.000*"thank" + 0.000*"support"
2019-02-25 18:38:04,668 : INFO : topic diff=0.103988, rho=0.277350
2019-02-25 18:38:21,058 : INFO : -8.372 per-word bound, 331.3 perplexity estimate based on a held-out corpus of 449 documents with 8187854 words
2019-02-25 18:38:21,058 : INFO : PROGRESS: pass 12, at document #449/449
2019-02-25 18:38:24,804 : INFO : topic #14 (0.05

In [56]:
maxTweets = 100000
tweetsDF[:maxTweets*10].groupby('twitter_account')['full_text'].apply(lambda x: "%s" % ' --\n '.join(x))

twitter_account
HuntingtonMayor    RT @medcouragement: Great Ted Talk by one of m...
Ianpgary           RT @NikoLusiani: What an opening salvo in the ...
IlhanMN            We are literally watching a manufactured crisi...
InhofePress        RT @tulsaworld: Sen. @JimInhofe defends milita...
JAHimes            @dyanna27 @RepJohnLarson @RepJoeCourtney @rosa...
JMVivancoHRW       El influyente congresista @RepMcGovern dice qu...
JPenaMelnyk        So honored to receive this award with @CherylK...
JRClemmons         I proudly joined the @TNJusticeCenter &amp; #h...
JacksonLeeTX18     This is wonderful news.  I am pleased that #ju...
JacksonforIndy     #Composure #GodsGrace https://t.co/CnwJUZfpWs ...
JakeCorman         HAPPENING NOW: The 103rd #PAFarmShow runs thro...
JanSchakowsky      RT @CNN: JUST IN: House Speaker Nancy Pelosi a...
JaneLubchenco      RT @tonybarnosky: Great postdoc in #Anthropoce...
JanetKavandi       RT @JimBridenstine: 50 years ago this Christma...
JaredPolis        

In [ ]:
new_doc = 'Practical Bayesian Optimization of Machine Learning Algorithms'
new_doc = prepare_text_for_lda(new_doc)
new_doc_bow = dictionary.doc2bow(new_doc)
print(new_doc_bow)
print(ldamodel.get_document_topics(new_doc_bow))

In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 3, id2word=dictionary, passes=15)
ldamodel.save('LDAModels/model3.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word=dictionary, passes=15)
ldamodel.save('LDAModels/model10.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

### pyLDAvis

In [ ]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('data/corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('LDAModels/model5.gensim')

In [ ]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [ ]:
lda3 = gensim.models.ldamodel.LdaModel.load('model3.gensim')
lda_display3 = pyLDAvis.gensim.prepare(lda3, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display3)

In [ ]:
lda10 = gensim.models.ldamodel.LdaModel.load('model10.gensim')
lda_display10 = pyLDAvis.gensim.prepare(lda10, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display10)